# 1.  Analyse ML Python
<img src="https://www.python.org/static/community_logos/python-logo.png">

## 1. Informations

In [35]:
import sys
sys.version

'3.6.9 |Anaconda, Inc.| (default, Jul 30 2019, 19:07:31) \n[GCC 7.3.0]'

In [36]:
import datetime
maintenant = datetime.datetime.now()
print('Date :', maintenant)

Date : 2021-01-08 11:59:49.411791


In [37]:
import pandas as pd
import numpy as np
import pickle
import os

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

In [38]:
import sklearn
print("I am using scikit learn", sklearn.__version__)

I am using scikit learn 0.22.2.post1


## 2. Chargement des données

Framingham Heart study dataset

https://www.kaggle.com/amanajmera1/framingham-heart-study-dataset

Attributes/columns:

- male: 0 = Female; 1 = Male
- age: Age at exam time
- education: 1 = Some High School; 2 = High School or GED; 3 = Some College or Vocational School; 4 = college
- currentSmoker: 0 = nonsmoker; 1 = smoker
- cigsPerDay: number of cigarettes smoked per day (estimated average)
- BPMeds: 0 = Not on Blood Pressure medications; 1 = Is on Blood Pressure medications
- prevalentStroke
- prevalentHyp
- diabetes: 0 = No; 1 = Yes
- totChol in mg/dL
- sysBP in mmHg
- diaBP in mmHg
- BMI: Body Mass Index calculated as: Weight (kg) / Height(meter-squared)
- heartRate: Beats/Min (Ventricular)
- glucose in mg/dL

- TenYearCHD - Did the person get heart disease in the 10 years study period?
label; 0 = No for heart disease, 1 = Yes for heart disease;


In [39]:
if os.path.exists("framingham.csv"):
  os.remove("framingham.csv")

In [40]:
!wget "https://raw.githubusercontent.com/retkowsky/WorkshopMLOps/master/framingham.csv"

--2021-01-08 11:59:52--  https://raw.githubusercontent.com/retkowsky/WorkshopMLOps/master/framingham.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.36.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.36.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 191803 (187K) [text/plain]
Saving to: ‘framingham.csv’

framingham.csv      100%[===================>] 187.31K  --.-KB/s    in 0.008s  

2021-01-08 11:59:52 (23.8 MB/s) - ‘framingham.csv’ saved [191803/191803]



In [41]:
df = pd.read_csv('framingham.csv')

In [42]:
df.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [43]:
df.shape

(4240, 16)

Préparation des données

In [44]:
smoke = (df['currentSmoker']==1)
df.loc[smoke,'cigsPerDay'] = df.loc[smoke,'cigsPerDay'].fillna(df.loc[smoke,'cigsPerDay'].mean())

In [45]:
df['BPMeds'].fillna(0, inplace = True)
df['glucose'].fillna(df.glucose.mean(), inplace = True)
df['totChol'].fillna(df.totChol.mean(), inplace = True)
df['education'].fillna(1, inplace = True)
df['BMI'].fillna(df.BMI.mean(), inplace = True)
df['heartRate'].fillna(df.heartRate.mean(), inplace = True)

## 3. Modélisation Random Forest

In [46]:
features = df.iloc[:,:-1]
result = df.iloc[:,-1]

Partitionnement des données

In [47]:
X_train, X_test, y_train, y_test = train_test_split(features, result, test_size = 0.2, random_state = 14)

Random Forest

In [48]:
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=2, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [49]:
# Create a selector object that will use the random forest classifier to identify
# features that have an importance of more than 0.12
sfm = SelectFromModel(clf, threshold=0.12)

sfm.fit(X_train, y_train)

SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                                 class_weight=None,
                                                 criterion='gini', max_depth=2,
                                                 max_features='auto',
                                                 max_leaf_nodes=None,
                                                 max_samples=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=100, n_jobs=None,
                                                 oob_score=False,
                                                 ra

In [50]:
feat_labels = list(features.columns.values)
for feature_list_index in sfm.get_support(indices=True):
    print(feat_labels[feature_list_index])

age
prevalentHyp
sysBP
glucose


In [51]:
importances = clf.feature_importances_
std = np.std([tree.feature_importances_ for tree in clf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

print("Feature ranking:")
for f in range(X_train.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

Feature ranking:
1. feature 1 (0.242214)
2. feature 10 (0.200736)
3. feature 14 (0.152858)
4. feature 7 (0.139117)
5. feature 11 (0.105004)
6. feature 0 (0.034898)
7. feature 12 (0.034277)
8. feature 4 (0.022181)
9. feature 5 (0.016595)
10. feature 8 (0.015724)
11. feature 9 (0.014800)
12. feature 13 (0.009745)
13. feature 2 (0.007321)
14. feature 6 (0.004530)
15. feature 3 (0.000000)


In [52]:
# With only imporant features. Can check X_important_train.shape[1]
X_important_train = sfm.transform(X_train)
X_important_test = sfm.transform(X_test)

In [53]:
%%time
clf_important = RandomForestClassifier(n_estimators=1000, random_state=0, n_jobs=-1)
clf_important.fit(X_important_train, y_train)

CPU times: user 5.12 s, sys: 718 ms, total: 5.84 s
Wall time: 2.74 s


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=-1, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

## 4. Métriques du modèle

In [54]:
predictions_y_4 = clf_important.predict(X_important_test)

print("Rapport de classification :")
print(classification_report(y_test, predictions_y_4))
print("")

print("Matrice de confusion :")
print(confusion_matrix(y_test, predictions_y_4))
print("")

# Under ROC curve
prob_y_4 = clf_important.predict_proba(X_important_test)
prob_y_4 = [p[1] for p in prob_y_4]
print("ROC =", roc_auc_score(y_test, prob_y_4))
print("")

print("Accuracy score =", accuracy_score(y_test, predictions_y_4))

Rapport de classification :
              precision    recall  f1-score   support

           0       0.86      0.96      0.91       724
           1       0.28      0.09      0.13       124

    accuracy                           0.83       848
   macro avg       0.57      0.52      0.52       848
weighted avg       0.77      0.83      0.79       848


Matrice de confusion :
[[695  29]
 [113  11]]

ROC = 0.6760102922830155

Accuracy score = 0.8325471698113207


## 5. Export du modèle de ML

In [55]:
os.makedirs('./sorties', exist_ok=True)

In [56]:
import datetime
maintenant = str(datetime.datetime.now())
print(maintenant)

2021-01-08 11:59:59.991075


In [57]:
filename = './sorties/modelenotebook-'+maintenant+'.pkl'

In [58]:
pickle.dump(clf_important, open(filename, 'wb'))
print("OK")

OK


In [59]:
!ls ./sorties/modelenotebook*.pkl -l

-rwxrwxrwx 1 root root 86129790 Jan  8 09:30 ./sorties/modelenotebook-2021-01-08 09:30:16.530399.pkl
-rwxrwxrwx 1 root root 86129790 Jan  8 12:00 ./sorties/modelenotebook-2021-01-08 11:59:59.991075.pkl


## 6. Tests du modèle

In [60]:
loaded_model = pickle.load(open(filename, 'rb'))

In [61]:
age = 61
prevalentHyp = 1
sysBP = 150
glucose = 103
 
print("Résultat :", loaded_model.predict([[age, prevalentHyp, sysBP, glucose]]))

Résultat : [1]


In [62]:
age = 43
prevalentHyp = 1
sysBP = 180
glucose = 99
 
print("Résultat :", loaded_model.predict([[age, prevalentHyp, sysBP, glucose]]))

Résultat : [0]


In [63]:
results = loaded_model.predict([[61, 1, 150, 103],[43, 1, 180, 99],[63,0,138,85]])
print("Résultats : ", results)

Résultats :  [1 0 1]


> Fin